In [7]:
!pip install SPARQLWrapper

from SPARQLWrapper import SPARQLWrapper, JSON
import ssl
import csv
import os

In [2]:
def sparl_query_setting(query, endpoint):# set the endpoint 
  sparql = SPARQLWrapper(endpoint)
  # set the query
  sparql.setQuery(query)
  # set the returned format
  sparql.setReturnFormat(JSON)
  # get the results
  results = sparql.query().convert()
  return results

In [8]:

def sample_selection(csv_to_create, query, endpoint, var_list): 
  with open(csv_to_create, mode='a') as my_file: 
    my_file = csv.writer(my_file, delimiter=',', quotechar='"', quoting=csv.QUOTE_ALL)
    if os.stat(csv_to_create).st_size == 0:
        my_file.writerow(var_list)
    res = sparl_query_setting(query, endpoint)
    for result in res["results"]["bindings"]:
      result_list = []
      for var in var_list: 
        if var not in result: 
          new_var = "none"
        else:
          new_var = result[var]["value"]
        result_list.append(new_var)
      my_file.writerow(result_list)



In [4]:
finland = "http://ldf.fi/mufi/sparql"
cervantes = "http://data.cervantesvirtual.com/sparql"
es_cultura= "https://datos.bne.es/sparql"
nl_spain = "http://cultura.linkeddata.es/sparql"
nd_hungary = "https://lod.sztaki.hu/sparql"
smithsonian = "https://api.triplydb.com/datasets/smithsonian/american-art-museum/services/american-art-museum/sparql"
american_lod = "https://data.americanartcollaborative.org/sparql"
dbpedia = "https://dbpedia.org/sparql"
yago = "https://yago-knowledge.org/sparql/query"
sari_endpoint= "https://rds.swissartresearch.net/sparql"
arco_endpoint = "https://dati.beniculturali.it/sparql"
wd_endpoint = "https://query.wikidata.org/bigdata/namespace/wdq/sparql"
zeri_endpoint = "http://data.fondazionezeri.unibo.it/sparql"
mmm_endpoint = "http://ldf.fi/mmm/sparql"
nomisma_endpoint = "http://nomisma.org/query"
b3k_endpoint = "http://lod.b3kat.de/doc/sparql-endpoint"
bnf_endpoint = "https://data.bnf.fr/sparql"
euro = "http://sparql.europeana.eu/"
arco= "https://dati.cultura.gov.it/sparql"

# SEMANTIC VALIDITY SAMPLE

In [ ]:
def makeatsv(filename, results, artwork, subject):
  with open(filename, 'wt') as out_file:
    tsv_writer = csv.writer(out_file, delimiter='\t')
    tsv_writer.writerow(['artwork', 'subject'])
    for result in results["results"]["bindings"]:
      tsv_writer.writerow([result[artwork]["value"], result[subject]["value"]])

## DBpedia

Take 50 random artwork-subject couples using the dbp:subject property
Take 50 random artwork-subject couples using the dct:subject property




In [ ]:
qmdb1 = '''
prefix dbo: <http://dbpedia.org/ontology/>
prefix dct: <http://purl.org/dc/terms/>
prefix dbp: <http://dbpedia.org/property/>
SELECT distinct ?artwork ?subject WHERE {
?artwork a dbo:Artwork;
dbp:subject ?subject .
}ORDER BY RAND()
LIMIT 50'''

qmdb2 = '''prefix dbo: <http://dbpedia.org/ontology/>
prefix dct: <http://purl.org/dc/terms/>
prefix dbp: <http://dbpedia.org/property/>
SELECT distinct ?artwork ?subject WHERE {
?artwork a dbo:Artwork;
dct:subject ?subject .
}ORDER BY RAND()
LIMIT 50'''

resmdb1 = sparl_query_setting(qmdb1, dbpedia)
resmdb2 = sparl_query_setting(qmdb2, dbpedia)

In [ ]:
import csv

In [ ]:
with open('dbpediasem.tsv', 'wt') as out_file:
    tsv_writer = csv.writer(out_file, delimiter='\t')
    tsv_writer.writerow(['artwork', 'subject'])
    for result in resmdb1["results"]["bindings"]:
      tsv_writer.writerow([result["artwork"]["value"], result["subject"]["value"]])
    for result2 in resmdb2["results"]["bindings"]:
      tsv_writer.writerow([result2["artwork"]["value"], result2["subject"]["value"]])

## Europeana

Take all the painting-subject triples (dc:subject)
Take all the sculpture-subject triples (dc:subject)

Randomly sample 100 triples from these two


In [ ]:
qeuro1paint = '''PREFIX ore: <http://www.openarchives.org/ore/terms/>
PREFIX edm: <http://www.europeana.eu/schemas/edm/>

select  distinct ?CHO ?sub where {
             ?s a <http://www.w3.org/2004/02/skos/core#Concept> ;
<http://www.w3.org/2004/02/skos/core#broader>* <http://vocab.getty.edu/aat/300033618> .
?CHO dc:type ?s ;
dc:subject ?sub 
}
'''

qeuro1sculpt= '''PREFIX ore: <http://www.openarchives.org/ore/terms/>
PREFIX edm: <http://www.europeana.eu/schemas/edm/>

select  distinct ?CHO ?sub where {
             ?s a <http://www.w3.org/2004/02/skos/core#Concept> ;
              <http://www.w3.org/2004/02/skos/core#broader>* <http://vocab.getty.edu/aat/300047090> .
?CHO dc:type ?s ;
dc:subject ?sub }'''

reseuro1p = sparl_query_setting(qeuro1paint, euro)
reseuro1s = sparl_query_setting(qeuro1sculpt, euro)

In [ ]:
paintings = []
for res in reseuro1p["results"]["bindings"]:
  artwork = res["CHO"]["value"]
  sub = res["sub"]["value"]
  paintings.append((artwork, sub))
print(len(paintings))
for res in reseuro1s["results"]["bindings"]:
  artwork = res["CHO"]["value"]
  sub = res["sub"]["value"]
  paintings.append((artwork, sub))
print(len(paintings))



9273
10690


In [ ]:
from random import sample
sampledartworks = sample(paintings, 100)

In [ ]:
print(sampledartworks[:10])

[('http://data.europeana.eu/proxy/provider/2023006/24062A51_priref_16880', 'schilderingen'), ('http://data.europeana.eu/proxy/provider/2023006/24062A51_priref_16309', 'schilderingen'), ('http://data.europeana.eu/proxy/provider/90402/SK_A_3758', 'Batavia'), ('http://data.europeana.eu/proxy/provider/2048077/Athena_Plus_ProvidedCHO_Bildarchiv_Foto_Marburg_obj_20395264_fmd458543', 'die zwölf Apostel (als Gruppe), in der Regel mit Büchern oder Schriftrollen (nicht im biblischen Kontext), mit Paulus statt Judas'), ('http://data.europeana.eu/proxy/provider/2023009/23096B51_priref_10626', 'fiets'), ('http://data.europeana.eu/proxy/provider/2048077/Athena_Plus_ProvidedCHO_Bildarchiv_Foto_Marburg_obj_20318319_fmd455670', 'Musikinstrumente; Gruppe von Musikinstrumenten'), ('http://data.europeana.eu/proxy/provider/90402/SK_A_742', 'http://iconclass.org/25F29(ARMADILLO)'), ('http://data.europeana.eu/proxy/provider/2026116/Partage_Plus_ProvidedCHO_Bildarchiv_Foto_Marburg_t2_20065140_fmd474022', 'htt

In [ ]:
import csv
with open("europeanasemantic.tsv", "wt") as out_file:
  twriter = csv.writer(out_file, delimiter="\t")
  twriter.writerow(["artwork", "subject"])
  for artsub in sampledartworks:
    art = artsub[0]
    sub = artsub[1]
    twriter.writerow([art, sub])

## Yago

In [ ]:
qy1 = '''
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX schema: <http://schema.org/>

SELECT distinct ?artwork ?sub
WHERE {
  VALUES ?art {schema:Painting schema:Sculpture}
  ?artwork rdf:type ?art ;
  schema:about ?sub 
}ORDER BY RAND()
LIMIT 100'''
ry1 = sparl_query_setting(qy1, yago)

In [ ]:
makeatsv("yago.tsv", ry1, "artwork", "sub")

## ND HUNGARY

500 results per query max, no random

Do 100 queries for artwork subject with random offset from 1 to 39441 
(total number of subjects)


In [ ]:

querynd = '''SELECT COUNT(?sub) as ?tot WHERE {
?URI <http://purl.org/dc/terms/type> dcmitype:Image ;
<http://purl.org/dc/terms/subject> ?sub .
 }'''

rndh = sparl_query_setting(querynd, nd_hungary)
for result in rndh["results"]["bindings"]:
  print(result["tot"]["value"])


39441


In [ ]:
import random
resultsnd = set()
queryndsample = '''SELECT ?URI ?sub WHERE {
?URI <http://purl.org/dc/terms/type> dcmitype:Image ;
<http://purl.org/dc/terms/subject> ?sub .
 }LIMIT 1
 OFFSET @@n@@'''
while len(resultsnd) < 100:
  randomn = random.randint(0, 39440)
  queryndrandom = queryndsample.replace("@@n@@", str(randomn))
  rndhsem = sparl_query_setting(queryndrandom, nd_hungary)
  for res in rndhsem["results"]["bindings"]:
    art = res["URI"]["value"]
    sub = res["sub"]["value"]
    resultsnd.add((art, sub))


In [ ]:
len(resultsnd)

100

In [ ]:
resultsnd

{('http://lod.sztaki.hu/data/item/10060', 'fa'),
 ('http://lod.sztaki.hu/data/item/10131', 'emberábrázolás'),
 ('http://lod.sztaki.hu/data/item/10142', 'magyar'),
 ('http://lod.sztaki.hu/data/item/10282', 'fénykép'),
 ('http://lod.sztaki.hu/data/item/10507', 'eredeti'),
 ('http://lod.sztaki.hu/data/item/10553', 'grafika'),
 ('http://lod.sztaki.hu/data/item/10589', 'magyar'),
 ('http://lod.sztaki.hu/data/item/10610', 'eredeti'),
 ('http://lod.sztaki.hu/data/item/10629', 'rézkarc'),
 ('http://lod.sztaki.hu/data/item/10841', 'emberábrázolás'),
 ('http://lod.sztaki.hu/data/item/10926', 'grafika'),
 ('http://lod.sztaki.hu/data/item/11113', 'magyar'),
 ('http://lod.sztaki.hu/data/item/11147', '1973-1989'),
 ('http://lod.sztaki.hu/data/item/11175', 'grafika'),
 ('http://lod.sztaki.hu/data/item/11219', 'rézkarc'),
 ('http://lod.sztaki.hu/data/item/11306', 'rézkarc'),
 ('http://lod.sztaki.hu/data/item/11327', 'akt'),
 ('http://lod.sztaki.hu/data/item/11347', 'ceruzarajz'),
 ('http://lod.sztaki.

In [ ]:
with open("nd_hungarysem.tsv", "wt") as out_file:
  twriter = csv.writer(out_file, delimiter="\t")
  twriter.writerow(["artwork", "subject"])
  for artsub in resultsnd:
    art = artsub[0]
    sub = artsub[1]
    twriter.writerow([art, sub])

## Zeri 

In [ ]:
# zeri
variables = ["art", "art_label", "subj", "subj_label"]
zeri_query = """

PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
# PREFIX oaentry: <http://purl.org/emmedi/oaentry/>
PREFIX crm: <http://www.cidoc-crm.org/cidoc-crm/>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX fabio: <http://purl.org/spar/fabio/>

SELECT distinct ?art (sample(?a_label) as ?art_label) (sample(?s) as ?subj) ?subj_label
WHERE {
  ?art a fabio:ArtisticWork. 
    ?art fabio:hasSubjectTerm ?s; rdfs:label ?a_label.
  ?s rdfs:label ?subj_label.
  }
  GROUP BY ?art ?subj_label
  ORDER BY RAND()
  LIMIT 100
  """


zeri_recall = sample_selection("semantic_validity_zeri.csv", zeri_query, zeri_endpoint, variables)

## Nomisma

In [ ]:
# nomisma
variables = ["art", "art_label", "subj", "subj_label"]
nomisma_query = """
PREFIX crm: <http://www.cidoc-crm.org/cidoc-crm/>
PREFIX dcmitype: <http://purl.org/dc/dcmitype/>
PREFIX dcterms: <http://purl.org/dc/terms/>
PREFIX foaf: <http://xmlns.com/foaf/0.1/>
PREFIX geo: <http://www.w3.org/2003/01/geo/wgs84_pos#>
PREFIX nm: <http://nomisma.org/id/>
PREFIX nmo: <http://nomisma.org/ontology#>
PREFIX org: <http://www.w3.org/ns/org#>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX skos: <http://www.w3.org/2004/02/skos/core#>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>


SELECT DISTINCT ?art ?art_label (sample(?s) as ?subj)  ?subj_label WHERE {
  
 {?coin nmo:hasObverse ?art; skos:prefLabel ?art_label} UNION {?coin nmo:hasReverse ?art; skos:prefLabel ?art_label}
  ?art nmo:hasPortrait ?s.
  ?s skos:prefLabel ?subj_label.
  FILTER(lang(?art_label)='en')
  FILTER(lang(?subj_label)='en')
  
} GROUP BY ?art ?art_label ?subj_label
  ORDER BY RAND()
  LIMIT 100


"""


nomisma_recall = sample_selection("semantic_validity_nomisma.csv", nomisma_query, nomisma_endpoint, variables)

## Wikidata

In [ ]:
# wikidata
variables = ["art", "s"]
wd_query1 = """

SELECT  DISTINCT ?art ?s WHERE {
  # SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en". }
  {?art wdt:P31 wd:Q3305213} UNION {?art wdt:P31 wd:Q860861}
  ?art wdt:P180 ?s.
  
} 
  ORDER BY RAND()

  LIMIT 34


"""
wd_query2 = """

SELECT  DISTINCT ?art ?s WHERE {
  # SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en". }
  {?art wdt:P31 wd:Q3305213} UNION {?art wdt:P31 wd:Q860861}
  ?art wdt:P921 ?s.
  
} 
  ORDER BY RAND()

  LIMIT 33


"""
wd_query3 = """

SELECT  DISTINCT ?art ?s WHERE {
  # SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en". }
  {?art wdt:P31 wd:Q3305213} UNION {?art wdt:P31 wd:Q860861}
  ?art wdt:P1257 ?s.
  
} 
  ORDER BY RAND()

  LIMIT 33


"""

recall1 = sample_selection("semantic_validity3_wd.csv", wd_query1, wd_endpoint, variables)
recall2 = sample_selection("semantic_validity3_wd.csv", wd_query2, wd_endpoint, variables)


In [ ]:
triples_list = []

wd_query = """

SELECT DISTINCT ?art ?s WHERE {
  # SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en". }
  {?art wdt:P31 wd:Q3305213} UNION {?art wdt:P31 wd:Q860861}
  ?art wdt:P1257 ?s.
  
} 
  ORDER BY RAND()



"""
res = sparl_query_setting(wd_query, wd_endpoint)

for result in res["results"]["bindings"]:
    triple = (result["art"]["value"], result["s"]["value"])
    print(triple)
    triples_list.append(triple)

Output streaming troncato alle ultime 5000 righe.
('http://www.wikidata.org/entity/Q28002591', '46C29(+63)')
('http://www.wikidata.org/entity/Q28002669', '46C29(+63)')
('http://www.wikidata.org/entity/Q28002683', '46C29(+63)')
('http://www.wikidata.org/entity/Q28002686', '46C29(+63)')
('http://www.wikidata.org/entity/Q28002086', '41D2663(+8)')
('http://www.wikidata.org/entity/Q28002113', '31D15(+822)')
('http://www.wikidata.org/entity/Q28002117', '31D15(+822)')
('http://www.wikidata.org/entity/Q28002136', '43A(+2)')
('http://www.wikidata.org/entity/Q28002628', '43A35')
('http://www.wikidata.org/entity/Q28002910', '31D15(+822)')
('http://www.wikidata.org/entity/Q28002912', '31D14(+812)')
('http://www.wikidata.org/entity/Q28002925', '41D222(+8)')
('http://www.wikidata.org/entity/Q28003037', '43B312')
('http://www.wikidata.org/entity/Q28003175', '34B2332(+982)')
('http://www.wikidata.org/entity/Q28003206', '31D15(+822)')
('http://www.wikidata.org/entity/Q28003233', '32B34')
('http://www.w

In [ ]:
from random import sample
sampledresults = sample(triples_list, 33)
print(len(sampledresults))

33


In [ ]:

with open("semantic_validity3_wd.csv", mode='a') as my_file: 
  my_file = csv.writer(my_file, delimiter=',', quotechar='"', quoting=csv.QUOTE_ALL)
  for triple in sampledresults:
      my_file.writerow(triple)

## Sari

In [ ]:
artwork_types= """<https://d-nb.info/gnd/4046277-8> <https://d-nb.info/gnd/4127900-1> <https://d-nb.info/gnd/4036042-8> <https://d-nb.info/gnd/4010427-8> <https://d-nb.info/gnd/4124432-1> <https://d-nb.info/gnd/4123412-1> <https://d-nb.info/gnd/4131808-0> <https://d-nb.info/gnd/4124432-1> <https://d-nb.info/gnd/4114333-4> <https://d-nb.info/gnd/4033760-1> <https://d-nb.info/gnd/4002511-1> <https://d-nb.info/gnd/4627787-0> <https://d-nb.info/gnd/4029783-4> <https://d-nb.info/gnd/4006627-7> <https://d-nb.info/gnd/4064542-3> <https://d-nb.info/gnd/4229053-3>"""
variables = ["art", "art_label", "subj"]

In [ ]:
# sari. No labels available for subjects. 

sari_query1 = """
PREFIX crm: <http://www.cidoc-crm.org/cidoc-crm/>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX getty: <http://vocab.getty.edu/ontology#>
PREFIX gndo: <https://d-nb.info/standards/elementset/gnd#>
  
SELECT DISTINCT ?art ?art_label (sample(?s) as ?subj) WHERE {
VALUES ?type {"""+artwork_types+"""}
  ?art gndo:formOfWorkAndExpression ?type; 
    gndo:preferredNameForTheWork ?art_label; 
    gndo:topic ?s. # gndo:gndSubjectCategory
   # FILTER(lang(?art_label)='en')

  
} GROUP BY ?art ?art_label ?subj
  ORDER BY RAND()
  LIMIT 50


"""
sari_query2 = """
PREFIX gndo: <https://d-nb.info/standards/elementset/gnd#>
  
SELECT DISTINCT ?art ?art_label (sample(?s) as ?subj) WHERE {
VALUES ?type {"""+artwork_types+"""}
  ?art gndo:formOfWorkAndExpression ?type; 
    gndo:preferredNameForTheWork ?art_label; 
    gndo:gndSubjectCategory ?s. # gndo:gndSubjectCategory
   # FILTER(lang(?art_label)='en')

  
} GROUP BY ?art ?art_label ?subj
  ORDER BY RAND()
  LIMIT 94


"""


recall1 = sample_selection("semantic_validity_sari.csv", sari_query1, sari_endpoint, variables)
recall2 = sample_selection("semantic_validity_sari.csv", sari_query2, sari_endpoint, variables)

## Arco
Since we have 5 relations identifying a subject, we select 20 artworks/subjects per relation, for a total of 100 artworks/subjects. 

In [10]:
import csv
with open('Arco_semantic_validity.csv', mode='w') as my_file:
  arco_file = csv.writer(my_file, delimiter=',', quotechar='"', quoting=csv.QUOTE_ALL)
  arco_file.writerow(['Artwork', 'Artwork_label', "Subject", 'Subject_label']) 
  

In [11]:
relations = ["arco-cd:hasSubject",	"arco-cd:iconclassCode", "arco-cd:subject", "dc:subject"]
# this query can sometimes return server error
# open csv, store the results of every iteration there. 
with open('Arco_semantic_validity.csv', mode='a') as my_file: 
  my_file = csv.writer(my_file, delimiter=',', quotechar='"', quoting=csv.QUOTE_ALL)

  for rel in relations: 
    arco_query = """

    PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
    PREFIX arco-cd: <https://w3id.org/arco/ontology/context-description/>
    PREFIX arco-dd: <https://w3id.org/arco/ontology/denotative-description/>
    PREFIX arco: <https://w3id.org/arco/ontology/arco/>
    PREFIX dc: <http://purl.org/dc/elements/1.1/>

    SELECT distinct ?art (sample(?art_label) as ?a_label) ?subj ?subj_label
    WHERE {
    ?art a arco:HistoricOrArtisticProperty. 
     ?art """+rel+""" ?subj; rdfs:label ?art_label.
    OPTIONAL {?subj rdfs:label ?subj_label}.
   # FILTER(lang(?art_label)='en')
   # FILTER(lang(?subj_label)='en')
    }
    GROUP BY ?art ?subj ?subj_label
    ORDER BY RAND()
    LIMIT 20
    """


    res = sparl_query_setting(arco_query, arco_endpoint)
    
    # manipulate the result
    for result in res["results"]["bindings"]:
      print(result)
      if "subj_label" not in result: 
        subj_label = "none"
      else:
        subj_label = result["subj_label"]["value"]
      my_file.writerow([result["art"]["value"], result["a_label"]["value"], result["subj"]["value"], subj_label])



EndPointInternalError: ignored

In [ ]:
variables = ["art", "a_label", "subj", "subj_label"]

iconogr_apparatus_query = """
    PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
    PREFIX arco-cd: <https://w3id.org/arco/ontology/context-description/>
    PREFIX arco-dd: <https://w3id.org/arco/ontology/denotative-description/>
    PREFIX arco: <https://w3id.org/arco/ontology/arco/>
    PREFIX dc: <http://purl.org/dc/elements/1.1/>

    SELECT distinct ?art (sample(?art_label) as ?a_label) ?subj ?subj_label
    WHERE {
      VALUES ?rel {arco-cd:hasSubject arco-cd:iconclassCode arco-cd:subject dc:subject}
    ?art a arco:HistoricOrArtisticProperty; 
      rdfs:label ?art_label. 
     ?art arco-dd:hasIconographicOrDecorativeApparatus ?iconogr.
     ?iconogr ?rel ?subj.
    OPTIONAL {?subj rdfs:label ?subj_label}.
    FILTER NOT EXISTS {?iconogr a arco-dd:DecorativeApparatus}
   # FILTER(lang(?art_label)='en')
   # FILTER(lang(?subj_label)='en')
    }
    GROUP BY ?art ?subj ?subj_label
    ORDER BY RAND()
    LIMIT 20
    """
recall = sample_selection("semantic_validity_arco.csv", iconogr_apparatus_query, arco_endpoint, variables)

# COLUMN COMPLETENESS SAMPLE

## ARCO

select 100 random works of art that have at least one of the subject properties

then create a values query to group together all the subjects/iconographic elements associated with them

In [ ]:
query100randomarco = '''PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX arco-cd: <https://w3id.org/arco/ontology/context-description/>
PREFIX arco-dd: <https://w3id.org/arco/ontology/denotative-description/>
PREFIX arco: <https://w3id.org/arco/ontology/arco/>
PREFIX datal: <https://w3id.org/arco/resource/Lombardia/HistoricOrArtisticProperty/>
PREFIX datas: <https://w3id.org/arco/resource/SARDEGNA/HistoricOrArtisticProperty/>
PREFIX data: <https://w3id.org/arco/resource/HistoricOrArtisticProperty/>


SELECT DISTINCT ?art where {
?art a arco:HistoricOrArtisticProperty ;
arco-cd:iconclassCode | dc:subject | arco-cd:subject | arco-dd:hasIconographicOrDecorativeApparatus ?subj .
} ORDER BY RAND ()
LIMIT 100'''

arco100randomresults = sparl_query_setting(query100randomarco, arco)

arcolist = []
for el in arco100randomresults["results"]["bindings"]:
  uri = str(el["art"]["value"])
  uri = "<" + uri + ">"
  arcolist.append(uri)
stringforvalues = " ".join(arcolist)


In [ ]:
stringforvalues[:2000]

'<https://w3id.org/arco/resource/HistoricOrArtisticProperty/0900014470> <https://w3id.org/arco/resource/HistoricOrArtisticProperty/0900341487> <https://w3id.org/arco/resource/HistoricOrArtisticProperty/1500662743> <https://w3id.org/arco/resource/HistoricOrArtisticProperty/0800202761> <https://w3id.org/arco/resource/HistoricOrArtisticProperty/1300144187> <https://w3id.org/arco/resource/HistoricOrArtisticProperty/2000052548> <https://w3id.org/arco/resource/HistoricOrArtisticProperty/1600047946> <https://w3id.org/arco/resource/HistoricOrArtisticProperty/0500447476> <https://w3id.org/arco/resource/HistoricOrArtisticProperty/0900329809> <https://w3id.org/arco/resource/HistoricOrArtisticProperty/0700008507> <https://w3id.org/arco/resource/HistoricOrArtisticProperty/1100253326> <https://w3id.org/arco/resource/HistoricOrArtisticProperty/1600183135> <https://w3id.org/arco/resource/HistoricOrArtisticProperty/2000065321> <https://w3id.org/arco/resource/HistoricOrArtisticProperty/0500446046> <http

In [ ]:
querycolumnarcowithrandomvalues = '''
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX arco-cd: <https://w3id.org/arco/ontology/context-description/>
PREFIX arco-dd: <https://w3id.org/arco/ontology/denotative-description/>
PREFIX arco: <https://w3id.org/arco/ontology/arco/>
PREFIX datal: <https://w3id.org/arco/resource/Lombardia/HistoricOrArtisticProperty/>
PREFIX datas: <https://w3id.org/arco/resource/SARDEGNA/HistoricOrArtisticProperty/>
PREFIX data: <https://w3id.org/arco/resource/HistoricOrArtisticProperty/>


SELECT DISTINCT ?art (GROUP_CONCAT(?subj; separator = " @ " ) as ?subjects) where {
VALUES ?art { '''+ stringforvalues +''' }
?art arco-cd:iconclassCode | dc:subject | arco-cd:subject | arco-dd:hasIconographicOrDecorativeApparatus ?subj .
}'''

resultscolumnsarco = sparl_query_setting(querycolumnarcowithrandomvalues, arco)


In [ ]:
import csv
makeatsv("arcocolumn.tsv", resultscolumnsarco, "art", "subjects")

## DBpedia

use built in order by rand()

In [ ]:
qmdb2 = '''
prefix dbo: <http://dbpedia.org/ontology/>
prefix dct: <http://purl.org/dc/terms/>
prefix dbp: <http://dbpedia.org/property/>
SELECT ?artwork (group_concat(distinct ?subject1; separator=" @ ") as ?subjects1) (group_concat(distinct ?subject2; separator = " @ ") as ?subjects2) WHERE {
?artwork a dbo:Artwork;
dct:subject ?subject1;
dbp:subject ?subject2 . }
ORDER BY RAND()
LIMIT 100'''

rescoldb = sparl_query_setting(qmdb2, dbpedia)



In [ ]:
with open("dbpedia_colc.tsv", "wt", encoding="utf-8-sig") as out_file:
  twriter = csv.writer(out_file, delimiter="\t")
  twriter.writerow(["artwork", "prop1", "prop2"])
  for res in rescoldb["results"]["bindings"]:
    art = res["artwork"]["value"]
    sub1 = res["subjects1"]["value"]
    sub2 = res["subjects2"]["value"]
    twriter.writerow([art, sub1, sub2]) 

## Europeana

take all the paintings - groupconcat(subjects) couples

take all the sculptures - groupconcat(subjects) couples

take a sample of 100 from the total

In [ ]:
euroqueryp = '''
PREFIX ore: <http://www.openarchives.org/ore/terms/>
PREFIX edm: <http://www.europeana.eu/schemas/edm/>

select  ?CHO (GROUP_CONCAT(distinct ?sub ;separator=" @ ") as ?subjects) where {
             ?s a <http://www.w3.org/2004/02/skos/core#Concept> ;
<http://www.w3.org/2004/02/skos/core#broader>* <http://vocab.getty.edu/aat/300033618> .
?CHO dc:type ?s ;
dc:subject ?sub . }'''

euroquerysc = '''
PREFIX ore: <http://www.openarchives.org/ore/terms/>
PREFIX edm: <http://www.europeana.eu/schemas/edm/>

select  ?CHO (GROUP_CONCAT(distinct ?sub ;separator=" @ ") as ?subjects) where {
             ?s a <http://www.w3.org/2004/02/skos/core#Concept> ;
<http://www.w3.org/2004/02/skos/core#broader>* <http://vocab.getty.edu/aat/300047090> .
?CHO dc:type ?s ;
dc:subject ?sub . }'''

reseuro2p = sparl_query_setting(euroqueryp, euro)
reseuro2s = sparl_query_setting(euroquerysc, euro)

In [ ]:
paintings2 = []
for res in reseuro2p["results"]["bindings"]:
  art = res["CHO"]["value"]
  subjects = res["subjects"]["value"]
  paintings2.append((art, subjects))
print(len(paintings2))
for res in reseuro2s["results"]["bindings"]:
  art = res["CHO"]["value"]
  subjects = res["subjects"]["value"]
  paintings2.append((art, subjects))
print(len(paintings2))

3954
5250


In [ ]:
sampledartworks2 = sample(paintings2, 100)

In [ ]:
with open("europeanacolumn3.tsv", "wt", encoding="utf-8-sig") as out_file:
  twriter = csv.writer(out_file, delimiter="§")
  twriter.writerow(["artwork", "subject"])
  for artsub in sampledartworks2:
    art = artsub[0]
    sub = artsub[1]
    twriter.writerow([art, sub])

## Yago

In [ ]:
qy2 = '''
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX schema: <http://schema.org/>

SELECT distinct ?artwork (GROUP_CONCAT(distinct ?sub;separator = " @ ") as ?subjects)
WHERE {
  VALUES ?art {schema:Painting schema:Sculpture}
  ?artwork rdf:type ?art ;
  schema:about ?sub 
} GROUP BY ?artwork
ORDER BY RAND()
LIMIT 100'''

ry2 = sparl_query_setting(qy2, yago)

In [ ]:
makeatsv("yagocolumn.tsv", ry2, "artwork", "subjects")

## ND HUNGARY

In [ ]:
querynd2 = '''SELECT COUNT(distinct ?URI) as ?tot WHERE {
?URI <http://purl.org/dc/terms/type> dcmitype:Image ;
<http://purl.org/dc/terms/subject> ?sub .
 }'''

rndh2 = sparl_query_setting(querynd2, nd_hungary)
for result in rndh2["results"]["bindings"]:
  print(result["tot"]["value"])

6523


In [ ]:
resultsnd2 = set()
queryndsample = '''SELECT ?URI WHERE {
?URI <http://purl.org/dc/terms/type> dcmitype:Image ;
<http://purl.org/dc/terms/subject> ?sub .
 }LIMIT 1
 OFFSET @@n@@'''
while len(resultsnd2) < 100:
  randomn = random.randint(0, 6522)
  queryndrandom = queryndsample.replace("@@n@@", str(randomn))
  rndhsem = sparl_query_setting(queryndrandom, nd_hungary)
  for res in rndhsem["results"]["bindings"]:
    art = res["URI"]["value"]
    resultsnd2.add(art)


In [ ]:
resultsnd2

{'http://lod.sztaki.hu/data/item/6188',
 'http://lod.sztaki.hu/data/item/6215',
 'http://lod.sztaki.hu/data/item/6224',
 'http://lod.sztaki.hu/data/item/6231',
 'http://lod.sztaki.hu/data/item/6234',
 'http://lod.sztaki.hu/data/item/6350',
 'http://lod.sztaki.hu/data/item/6379',
 'http://lod.sztaki.hu/data/item/6402',
 'http://lod.sztaki.hu/data/item/6421',
 'http://lod.sztaki.hu/data/item/6423',
 'http://lod.sztaki.hu/data/item/6499',
 'http://lod.sztaki.hu/data/item/6678',
 'http://lod.sztaki.hu/data/item/6689',
 'http://lod.sztaki.hu/data/item/6690',
 'http://lod.sztaki.hu/data/item/6706',
 'http://lod.sztaki.hu/data/item/6708',
 'http://lod.sztaki.hu/data/item/6720',
 'http://lod.sztaki.hu/data/item/6723',
 'http://lod.sztaki.hu/data/item/6755',
 'http://lod.sztaki.hu/data/item/6761',
 'http://lod.sztaki.hu/data/item/6793',
 'http://lod.sztaki.hu/data/item/6916',
 'http://lod.sztaki.hu/data/item/6936',
 'http://lod.sztaki.hu/data/item/6949',
 'http://lod.sztaki.hu/data/item/6959',


In [ ]:
artworkssub = dict()
samplequery = '''
SELECT distinct ?sub where {
@@@uri@@@ <http://purl.org/dc/terms/subject> ?sub .
}
'''
for el in resultsnd2:
  artworkssub[el] = []
  queryel = "<" + el + ">"
  querynd = samplequery.replace("@@@uri@@@", queryel)
  rndhsem = sparl_query_setting(querynd, nd_hungary)
  for res in rndhsem["results"]["bindings"]:
    sub = res["sub"]["value"]
    artworkssub[el].append(sub)



In [ ]:
artworkssub

{'http://lod.sztaki.hu/data/item/6188': ['plakát',
  'eredeti',
  'Matejko, Theo (1893-1944)',
  'német',
  'műtárgy',
  'plakátművészet',
  '20. század első fele'],
 'http://lod.sztaki.hu/data/item/6215': ['eredeti',
  'férfi',
  'fényképek',
  '20. század eleje',
  'akt',
  'fotóművészet'],
 'http://lod.sztaki.hu/data/item/6224': ['olasz',
  'eredeti',
  'fénykép',
  'reneszánsz',
  'Raffaello di Giovanni Santi (1483-1520)',
  'festészet 16. sz. eleje',
  'fotóművészet',
  'műtárgyfotó 19. sz. vége'],
 'http://lod.sztaki.hu/data/item/6231': ['olasz',
  'eredeti',
  'fénykép',
  'reneszánsz',
  'freskó',
  'Correggio (1489-1534)',
  'fotóművészet',
  'műtárgyfotó 19. sz. vége',
  'festészet 16. sz. első harmada'],
 'http://lod.sztaki.hu/data/item/6234': ['olasz',
  'eredeti',
  'fénykép',
  'reneszánsz',
  'freskó',
  'Correggio (1489-1534)',
  'fotóművészet',
  'műtárgyfotó 19. sz. vége',
  'festészet 16. sz. első harmada'],
 'http://lod.sztaki.hu/data/item/6350': ['eredeti',
  'fény

In [ ]:
len(artworkssub)

100

In [ ]:
with open("ndhungarycol.tsv", "wt", encoding="utf-8-sig") as out_file:
  twriter = csv.writer(out_file, delimiter="\t")
  twriter.writerow(["artwork", "subjects"])
  for el in artworkssub:
    twriter.writerow([el, " @ ".join(artworkssub[el])])

## Zeri

In [ ]:
# zeri
variables = ["art", "subj_descr"]
zeri_query = """
PREFIX crm: <http://www.cidoc-crm.org/cidoc-crm/>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX fabio: <http://purl.org/spar/fabio/>


SELECT distinct ?art (GROUP_CONCAT(distinct ?subj_label; separator = " @ ") as ?subj_descr)
WHERE {
 
  ?art a fabio:ArtisticWork. 
    ?art fabio:hasSubjectTerm ?subj; rdfs:label ?art_label.
  ?subj rdfs:label ?subj_label.
}
 GROUP BY ?art
 ORDER BY RAND()
 LIMIT 100

"""


recall = sample_selection("column_completeness_zer.csv", zer_query, zeri_endpoint, variables)

## Nomisma

In [ ]:
# nomisma
variables = ["art", "subj_descr"]
nomisma_query = """
PREFIX crm: <http://www.cidoc-crm.org/cidoc-crm/>
PREFIX dcmitype: <http://purl.org/dc/dcmitype/>
PREFIX dcterms: <http://purl.org/dc/terms/>
PREFIX foaf: <http://xmlns.com/foaf/0.1/>
PREFIX geo: <http://www.w3.org/2003/01/geo/wgs84_pos#>
PREFIX nm: <http://nomisma.org/id/>
PREFIX nmo: <http://nomisma.org/ontology#>
PREFIX org: <http://www.w3.org/ns/org#>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX skos: <http://www.w3.org/2004/02/skos/core#>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>


SELECT DISTINCT ?art (GROUP_CONCAT(distinct ?subj_label; separator = " @ ") as ?subj_descr) WHERE {
  
 {?coin nmo:hasObverse ?art} UNION {?coin nmo:hasReverse ?art}
  ?art nmo:hasPortrait ?subj.
  ?subj skos:prefLabel ?subj_label.

  FILTER(lang(?subj_label)='en')
  
} GROUP BY ?art
ORDER BY RAND()
  LIMIT 100


"""


nomisma_recall = sample_selection("column_completeness_nomisma.csv", nomisma_query, nomisma_endpoint, variables)

## Wikidata

In [5]:
# select all the qualifiers of wdt:P180
wd_constraint = []
wd_query_constr = """

SELECT DISTINCT ?qual {
  wd:P180 p:P2302 [ ps:P2302 wd:Q21510851 ; pq:P2306 ?constr] .   
  ?constr wikibase:qualifier ?qual.                       
  SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en" }   
} ORDER BY ASC(xsd:integer(strafter(str(?prop), concat(str(wd:), "P"))))

"""

res = sparl_query_setting(wd_query_constr, wd_endpoint)

# manipulate the result
for result in res["results"]["bindings"]:
  constraint = "<"+result["qual"]["value"]+">"
  wd_constraint.append(constraint)

print(wd_constraint)

['<http://www.wikidata.org/prop/qualifier/P18>', '<http://www.wikidata.org/prop/qualifier/P21>', '<http://www.wikidata.org/prop/qualifier/P127>', '<http://www.wikidata.org/prop/qualifier/P137>', '<http://www.wikidata.org/prop/qualifier/P144>', '<http://www.wikidata.org/prop/qualifier/P149>', '<http://www.wikidata.org/prop/qualifier/P155>', '<http://www.wikidata.org/prop/qualifier/P156>', '<http://www.wikidata.org/prop/qualifier/P175>', '<http://www.wikidata.org/prop/qualifier/P176>', '<http://www.wikidata.org/prop/qualifier/P180>', '<http://www.wikidata.org/prop/qualifier/P186>', '<http://www.wikidata.org/prop/qualifier/P276>', '<http://www.wikidata.org/prop/qualifier/P304>', '<http://www.wikidata.org/prop/qualifier/P407>', '<http://www.wikidata.org/prop/qualifier/P459>', '<http://www.wikidata.org/prop/qualifier/P462>', '<http://www.wikidata.org/prop/qualifier/P518>', '<http://www.wikidata.org/prop/qualifier/P560>', '<http://www.wikidata.org/prop/qualifier/P585>', '<http://www.wikidata

In [12]:
# this query can sometimes return timeout error
constr_values = " ".join(wd_constraint)
variables = ["art", "subj1", "constr", "qualifier", "subj2", "subj3"]
wd_query_constr = """

PREFIX wd: <http://www.wikidata.org/entity/>
PREFIX pq: <http://www.wikidata.org/prop/qualifier/>
PREFIX ps: <http://www.wikidata.org/prop/statement/>
PREFIX p: <http://www.wikidata.org/prop/>

SELECT DISTINCT ?art (GROUP_CONCAT(distinct ?subj1 ; separator = " @ ") as ?subject1) ?subj_qualified (GROUP_CONCAT(distinct ?subj2; separator = " @ ") as ?subject2) (GROUP_CONCAT(distinct ?subj3; separator = " @ ") as ?subject3)  WHERE {
   VALUES ?constr {<http://www.wikidata.org/prop/qualifier/P18> <http://www.wikidata.org/prop/qualifier/P21> <http://www.wikidata.org/prop/qualifier/P127> <http://www.wikidata.org/prop/qualifier/P144> <http://www.wikidata.org/prop/qualifier/P149> <http://www.wikidata.org/prop/qualifier/P155> <http://www.wikidata.org/prop/qualifier/P156> <http://www.wikidata.org/prop/qualifier/P175> <http://www.wikidata.org/prop/qualifier/P176> <http://www.wikidata.org/prop/qualifier/P180> <http://www.wikidata.org/prop/qualifier/P186> <http://www.wikidata.org/prop/qualifier/P276> <http://www.wikidata.org/prop/qualifier/P304> <http://www.wikidata.org/prop/qualifier/P407> <http://www.wikidata.org/prop/qualifier/P459> <http://www.wikidata.org/prop/qualifier/P462> <http://www.wikidata.org/prop/qualifier/P518> <http://www.wikidata.org/prop/qualifier/P560> <http://www.wikidata.org/prop/qualifier/P585> <http://www.wikidata.org/prop/qualifier/P642> <http://www.wikidata.org/prop/qualifier/P710> <http://www.wikidata.org/prop/qualifier/P1011> <http://www.wikidata.org/prop/qualifier/P1012> <http://www.wikidata.org/prop/qualifier/P1107> <http://www.wikidata.org/prop/qualifier/P1114> <http://www.wikidata.org/prop/qualifier/P1120> <http://www.wikidata.org/prop/qualifier/P1132> <http://www.wikidata.org/prop/qualifier/P1269> <http://www.wikidata.org/prop/qualifier/P1310> <http://www.wikidata.org/prop/qualifier/P1340> <http://www.wikidata.org/prop/qualifier/P1346> <http://www.wikidata.org/prop/qualifier/P1354> <http://www.wikidata.org/prop/qualifier/P1419> <http://www.wikidata.org/prop/qualifier/P1480> <http://www.wikidata.org/prop/qualifier/P1535> <http://www.wikidata.org/prop/qualifier/P1545> <http://www.wikidata.org/prop/qualifier/P1552> <http://www.wikidata.org/prop/qualifier/P1591> <http://www.wikidata.org/prop/qualifier/P1594> <http://www.wikidata.org/prop/qualifier/P1620> <http://www.wikidata.org/prop/qualifier/P1684> <http://www.wikidata.org/prop/qualifier/P1810> <http://www.wikidata.org/prop/qualifier/P1884> <http://www.wikidata.org/prop/qualifier/P1932> <http://www.wikidata.org/prop/qualifier/P2210> <http://www.wikidata.org/prop/qualifier/P2241> <http://www.wikidata.org/prop/qualifier/P2283> <http://www.wikidata.org/prop/qualifier/P2634> <http://www.wikidata.org/prop/qualifier/P2677> <http://www.wikidata.org/prop/qualifier/P2868> <http://www.wikidata.org/prop/qualifier/P2913> <http://www.wikidata.org/prop/qualifier/P3629> <http://www.wikidata.org/prop/qualifier/P3680> <http://www.wikidata.org/prop/qualifier/P3828> <http://www.wikidata.org/prop/qualifier/P3831> <http://www.wikidata.org/prop/qualifier/P4878> <http://www.wikidata.org/prop/qualifier/P4895> <http://www.wikidata.org/prop/qualifier/P5102> <http://www.wikidata.org/prop/qualifier/P5422> <http://www.wikidata.org/prop/qualifier/P5961> <http://www.wikidata.org/prop/qualifier/P6022> <http://www.wikidata.org/prop/qualifier/P6568> <http://www.wikidata.org/prop/qualifier/P7380> <http://www.wikidata.org/prop/qualifier/P7452> <http://www.wikidata.org/prop/qualifier/P8031> <http://www.wikidata.org/prop/qualifier/P8032> <http://www.wikidata.org/prop/qualifier/P8503> <http://www.wikidata.org/prop/qualifier/P8822> <http://www.wikidata.org/prop/qualifier/P8839>}
   
   SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en". }
     {?art wdt:P31 wd:Q3305213} UNION {?art wdt:P31 wd:Q860861}
  ?art wdt:P921 ?subj2; wdt:P1257 ?subj3.  
  ?art p:P180 ?P180node . 
  ?P180node ps:P180 ?subj1. 
  OPTIONAL {
   
    ?P180node ?constr ?qualifier  
    BIND (CONCAT(?subj1, ?constr, ?qualifier) AS ?subj_qualified)
  }
} GROUP BY ?art ?subj_qualified
  # ORDER BY RAND()

  LIMIT 100
  

  """

recall = sample_selection("column_completeness_wd.csv", wd_query_constr, wd_endpoint, variables)



EndPointInternalError: ignored

# Interlinking of artworks via owl:sameAs or similar

## DBpedia

In [ ]:
# How many artworks

queryalldb = '''
prefix dbo: <http://dbpedia.org/ontology/>
prefix dct: <http://purl.org/dc/terms/>
prefix dbp: <http://dbpedia.org/property/>
SELECT (COUNT(distinct ?artwork) as ?tot) WHERE {
?artwork a dbo:Artwork }'''

queryalldbres = sparl_query_setting(queryalldb, dbpedia)

for result in queryalldbres["results"]["bindings"]:
  print(result["tot"]["value"])
  tot = result["tot"]["value"]

# How many artworks linked with owl:sameAs?
queryowldb = '''
prefix owl: <http://www.w3.org/2002/07/owl#>
prefix dbo: <http://dbpedia.org/ontology/>
prefix dct: <http://purl.org/dc/terms/>
prefix dbp: <http://dbpedia.org/property/>
SELECT (COUNT(distinct ?artwork) as ?tot) WHERE {
?artwork a dbo:Artwork;
owl:sameAs ?x }'''

queryowldbres = sparl_query_setting(queryowldb, dbpedia)
for result in queryowldbres["results"]["bindings"]:
  print(result["tot"]["value"])
  owl = result["tot"]["value"]

print(int(owl)/int(tot))

11678
11588
0.9922932008905635


In [ ]:
dbpediainterlinkingscore = 0.9922932008905635

## Europeana

In [ ]:
# All artworks in europeana
queryalleuro = '''
PREFIX ore: <http://www.openarchives.org/ore/terms/>
PREFIX edm: <http://www.europeana.eu/schemas/edm/>

select  (COUNT(distinct ?CHO) as ?tot) where {
             ?s a <http://www.w3.org/2004/02/skos/core#Concept> ;
<http://www.w3.org/2004/02/skos/core#broader>* <http://vocab.getty.edu/aat/300033618> .
?s2 a <http://www.w3.org/2004/02/skos/core#Concept>  ;
<http://www.w3.org/2004/02/skos/core#broader>* <http://vocab.getty.edu/aat/300047090> .
{?CHO dc:type ?s} UNION {?CHO dc:type ?s2}
}'''

resalle = sparl_query_setting(queryalleuro, euro)
for el in resalle["results"]["bindings"]:
  print(el["tot"]["value"])
  alleuro = el["tot"]["value"]
# All artworks in europeana related to another external artwork
queryrelatedeuro = '''PREFIX ore: <http://www.openarchives.org/ore/terms/>
PREFIX edm: <http://www.europeana.eu/schemas/edm/>

select  (count(distinct ?CHO) as ?tot) where {
             ?s a <http://www.w3.org/2004/02/skos/core#Concept> ;
<http://www.w3.org/2004/02/skos/core#broader>* <http://vocab.getty.edu/aat/300033618> .
?s2 a <http://www.w3.org/2004/02/skos/core#Concept>  ;
<http://www.w3.org/2004/02/skos/core#broader>* <http://vocab.getty.edu/aat/300047090> .
{?CHO dc:type ?s;
dc:relation|edm:relatedTo ?x} UNION {?CHO dc:type ?s2;
dc:relation|edm:relatedTo ?x}
}'''

resrelatede = sparl_query_setting(queryrelatedeuro, euro)
for el in resrelatede["results"]["bindings"]:
  print(el["tot"]["value"])
  eurorelated = el["tot"]["value"]

print(int(eurorelated)/int(alleuro))

13861
101
0.007286631556164779


In [ ]:
europeanainterscore = 0.007286631556164779

## Yago

In [ ]:
qy4all = '''
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX schema: <http://schema.org/>

SELECT (count(distinct ?artwork) as ?tot)
WHERE {
  VALUES ?art {schema:Painting schema:Sculpture}
  ?artwork rdf:type ?art }'''

ry4all = sparl_query_setting(qy4all, yago)

for el in ry4all["results"]["bindings"]:
  print(el["tot"]["value"])
  totalyago = int(el["tot"]["value"])

qy4owl = '''
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX schema: <http://schema.org/>
PREFIX owl: <http://www.w3.org/2002/07/owl#>

SELECT (count(distinct ?artwork) as ?tot)
WHERE {
  VALUES ?art {schema:Painting schema:Sculpture}
  ?artwork rdf:type ?art;
  owl:sameAs ?something }'''

ry4owl = sparl_query_setting(qy4owl, yago)
for el in ry4owl["results"]["bindings"]:
  totalowl = int(el["tot"]["value"])
  print(totalowl)

print(totalowl/totalyago)

29324
29324
1.0


In [ ]:
yagointerlinkingscore = 1

## ND Hungary

Does not have any owl:sameAs / related property

In [ ]:
listofproperties = '''PREFIX dc: <http://purl.org/dc/terms/>
SELECT distinct ?p WHERE { ?URI dc:type ?type;
?p ?o
FILTER (?type = "painting" || ?type  = "sculpture")} LIMIT 100'''

propertiesofndhungary = sparl_query_setting(listofproperties, nd_hungary)

for el in propertiesofndhungary["results"]["bindings"]:
  print(el["p"]["value"])

http://www.w3.org/1999/02/22-rdf-syntax-ns#type
http://www.w3.org/2000/01/rdf-schema#label
http://purl.org/dc/terms/title
http://purl.org/dc/terms/description
http://purl.org/dc/terms/date
http://purl.org/dc/terms/identifier
http://purl.org/dc/terms/format
http://purl.org/dc/terms/publisher
http://purl.org/dc/terms/creator
http://purl.org/dc/terms/type


In [ ]:
ndinterlinkingscore = 0

## Arco

In [ ]:
arco_query = """

PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX arco-cd: <https://w3id.org/arco/ontology/context-description/>
PREFIX arco: <https://w3id.org/arco/ontology/arco/>

SELECT (count (distinct ?s) as ?tot) 
WHERE {
?s a arco:HistoricOrArtisticProperty. 
?s  owl:sameAs | rdfs:seeAlso ?resource. 
}
"""



res = sparl_query_setting(arco_query, arco_endpoint)

# manipulate the result
for result in res["results"]["bindings"]:
    print(result["tot"]["value"])

5447


## Zeri

In [ ]:
zeri_query = """

PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX fabio: <http://purl.org/spar/fabio/>
PREFIX owl: <http://www.w3.org/2002/07/owl#>

SELECT (count (distinct ?artwork) as ?tot) WHERE {
  ?artwork a fabio:ArtisticWork; 
  owl:sameAs | rdfs:seeAlso ?resource. 
} 


"""

res = sparl_query_setting(zeri_query, zeri_endpoint)

# manipulate the result
for result in res["results"]["bindings"]:
    print(result["tot"]["value"])

11


## Nomisma

In [ ]:
nomisma_query = """
PREFIX crm: <http://www.cidoc-crm.org/cidoc-crm/>
PREFIX dcmitype: <http://purl.org/dc/dcmitype/>
PREFIX dcterms: <http://purl.org/dc/terms/>
PREFIX foaf: <http://xmlns.com/foaf/0.1/>
PREFIX geo: <http://www.w3.org/2003/01/geo/wgs84_pos#>
PREFIX nm: <http://nomisma.org/id/>
PREFIX nmo: <http://nomisma.org/ontology#>
PREFIX org: <http://www.w3.org/ns/org#>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX skos: <http://www.w3.org/2004/02/skos/core#>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
PREFIX owl: <http://www.w3.org/2002/07/owl#>


SELECT (count(distinct ?art) as ?tot_art) WHERE {
 
  {?coin nmo:hasObverse ?art} UNION {?coin nmo:hasReverse ?art}
  ?coin owl:sameAs ?resource. 
} 


"""

res = sparl_query_setting(nomisma_query, nomisma_endpoint)

# manipulate the result
for result in res["results"]["bindings"]:
  print(result["tot_art"]["value"])


0


## Wikidata
Since Wikidata don't use owl:sameAs to connect resources, but specific relations for every institution, we select xxx properties to evaluate interlinking. 

In [ ]:

wd_query = """
SELECT (count(distinct ?something) as ?tot) WHERE {
VALUES ?class { wd:Q3305213 wd:Q860861 }
?something wdt:P31 ?class ;
           wikibase:identifiers ?ident .
FILTER (?ident > 0)
}
"""

res = sparl_query_setting(wd_query, wd_endpoint)

# manipulate the result
for result in res["results"]["bindings"]:
  print(result["tot"]["value"])

414678


Sari

In [ ]:

sari_query = """
PREFIX crm: <http://www.cidoc-crm.org/cidoc-crm/>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX getty: <http://vocab.getty.edu/ontology#>
PREFIX gndo: <https://d-nb.info/standards/elementset/gnd#>

SELECT (count(distinct ?a) as ?tot) WHERE {
VALUES ?type{"""+artwork_types+"""}
  ?a a gndo:Work; 
     gndo:formOfWorkAndExpression ?type; 
  owl:sameAs ?resource. 


} 

"""

res = sparl_query_setting(sari_query, sari_endpoint)

# manipulate the result

for result in res["results"]["bindings"]:
 print(result["tot"]["value"])

338


# Reference to external taxonomy (Iconclass/Getty)

## DBpedia, Yago, ND Hungary, Museum Finland

In [ ]:
# not linked to iconclass or aat
dbpediataxonomyscore = 0
yagotaxonomyscore = 0
ndhungarytaxonomyscore = 0
museumfinlandtaxonomyscore = 0

## Europeana

In [ ]:
querytaxonomy = '''PREFIX ore: <http://www.openarchives.org/ore/terms/>
PREFIX edm: <http://www.europeana.eu/schemas/edm/>

select  (count(distinct ?CHO) as ?tot) where {
             ?s a <http://www.w3.org/2004/02/skos/core#Concept> ;
<http://www.w3.org/2004/02/skos/core#broader>* <http://vocab.getty.edu/aat/300033618> .
?s2 a <http://www.w3.org/2004/02/skos/core#Concept>  ;
<http://www.w3.org/2004/02/skos/core#broader>* <http://vocab.getty.edu/aat/300047090> .

{?CHO dc:type ?s;
dc:subject ?sub .
FILTER (STRSTARTS(str(?sub), "http://iconclass"))}

UNION 

{?CHO dc:type ?s2;
dc:subject ?sub .
FILTER (STRSTARTS(str(?sub), "http://iconclass"))}
}'''

querytaxonomyres = sparl_query_setting(querytaxonomy, euro)

for el in querytaxonomyres["results"]["bindings"]:
  print(el["tot"]["value"])
  europeanataxonomy = int(el["tot"]["value"])


print(europeanataxonomy/int(alleuro))
                            

905
0.06529110453791213


In [ ]:
europeanaiconclasstaxscore = 0.06529110453791213
europeanaaatscore = 1 #all the paintings and sculptures are retrieved using AAT

## Arco

In [ ]:
arco_query = """

PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX arco-cd: <https://w3id.org/arco/ontology/context-description/>
PREFIX arco: <https://w3id.org/arco/ontology/arco/>

SELECT (count (distinct ?s) as ?tot) 
WHERE {
?s a arco:HistoricOrArtisticProperty; 
  arco-cd:iconclassCode ?resource. 
}
"""



res = sparl_query_setting(arco_query, arco_endpoint)

# manipulate the result
for result in res["results"]["bindings"]:
    print(result["tot"]["value"])

261531


## Zeri

In [ ]:
zeri_query = """

PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX fabio: <http://purl.org/spar/fabio/>
PREFIX owl: <http://www.w3.org/2002/07/owl#>

SELECT (count (distinct ?artwork) as ?tot) WHERE {
  ?artwork rdf:type fabio:ArtisticWork; 
  fabio:hasSubjectTerm ?subj.
  ?subj rdfs:seeAlso ?resource. 
  FILTER (regex(str(?resource), "iconclass", "i"))
}


"""

res = sparl_query_setting(zeri_query, zeri_endpoint)

# manipulate the result
for result in res["results"]["bindings"]:
    print(result["tot"]["value"])

10943


## Nomisma
It has links only to Getty Vocabularies, none to Iconclass

In [ ]:
# subjects relation exploration

nomisma_query = """
PREFIX crm: <http://www.cidoc-crm.org/cidoc-crm/>
PREFIX dcmitype: <http://purl.org/dc/dcmitype/>
PREFIX dcterms: <http://purl.org/dc/terms/>
PREFIX foaf: <http://xmlns.com/foaf/0.1/>
PREFIX geo: <http://www.w3.org/2003/01/geo/wgs84_pos#>
PREFIX nm: <http://nomisma.org/id/>
PREFIX nmo: <http://nomisma.org/ontology#>
PREFIX org: <http://www.w3.org/ns/org#>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX skos: <http://www.w3.org/2004/02/skos/core#>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>


SELECT (count(distinct ?obj) as ?tot) WHERE {
 
  ?sub nmo:hasPortrait ?obj.
  ?obj skos:exactMatch ?obj2. 
  FILTER(regex(str(?obj2), "getty", "i"))
 
} 


"""

res = sparl_query_setting(nomisma_query, nomisma_endpoint)

# manipulate the result
for result in res["results"]["bindings"]:
  print(result["tot"]["value"])

39


Wikidata

In [ ]:
# wikidata. Number of paintings and sculptures having an iconclass code directly associated


wd_query = """

SELECT (count(distinct ?art) as ?tot) WHERE {
  {?art wdt:P31 wd:Q3305213} UNION {?art wdt:P31 wd:Q860861}
 
  VALUES ?rel {wdt:P1256 wdt:P5986}
{
  ?art (wdt:P180 | wdt:P921) ?subj.
  ?subj ?rel ?resource. 
  }
  UNION 
{
  ?art wdt:P1257 ?resource.
  
}
} 

"""

res = sparl_query_setting(wd_query, wd_endpoint)

# manipulate the result
for result in res["results"]["bindings"]:
  print(result["tot"]["value"])

74213


## SARI
We runned the same query looking for "Getty" and "Iconclass" presence in the results through a regex. 

In [ ]:

sari_query = """
PREFIX crm: <http://www.cidoc-crm.org/cidoc-crm/>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX getty: <http://vocab.getty.edu/ontology#>
PREFIX gndo: <https://d-nb.info/standards/elementset/gnd#>

SELECT (count (distinct ?a) as ?tot) WHERE {
VALUES ?type{"""+artwork_types+"""}
  ?a a gndo:Work; 
     gndo:formOfWorkAndExpression ?type; 
     ?rel ?resource.

FILTER(regex(str(?resource), "iconclass", "i"))


} LIMIT 100

"""

res = sparl_query_setting(sari_query, sari_endpoint)

# manipulate the result

for result in res["results"]["bindings"]:
 print(result["tot"]["value"])

0


# Intra-interlinking 

## DBpedia

In [ ]:
intrainterlinkingpotentialdb = '''prefix dbo: <http://dbpedia.org/ontology/>
prefix dct: <http://purl.org/dc/terms/>
prefix dbp: <http://dbpedia.org/property/>
SELECT (AVG(?no) AS ?avg) 
WHERE { 
 SELECT (COUNT(distinct ?instance) AS ?no) 
 WHERE { ?instance a dbo:Artwork ;
dbp:subject|dct:subject ?subject }
GROUP BY ?subject
}'''

iilpresdb = sparl_query_setting(intrainterlinkingpotentialdb, dbpedia)

for el in iilpresdb["results"]["bindings"]:
  print(el["avg"]["value"])
  iilpdbscore = el["avg"]["value"]

4.173305056663795


In [ ]:
subjectmorethan1db = '''prefix dbo: <http://dbpedia.org/ontology/>
prefix dct: <http://purl.org/dc/terms/>
prefix dbp: <http://dbpedia.org/property/>
SELECT (COUNT(distinct ?subject) as ?tot)
WHERE { FILTER (?no > 1) {
 SELECT ?subject (COUNT(distinct ?instance) AS ?no) 
 WHERE { ?instance a dbo:Artwork ;
dbp:subject|dct:subject ?subject }
GROUP BY ?subject }
}'''

allsubsuridb = '''prefix dbo: <http://dbpedia.org/ontology/>
prefix dct: <http://purl.org/dc/terms/>
prefix dbp: <http://dbpedia.org/property/>
 SELECT (COUNT(distinct ?subject) as ?tot)
 WHERE { ?instance a dbo:Artwork ;
dbp:subject|dct:subject ?subject
FILTER (isURI(?subject)) }'''

submorethan1dbres = sparl_query_setting(subjectmorethan1db, dbpedia)
allsubsuridbres = sparl_query_setting(allsubsuridb, dbpedia)

for el in submorethan1dbres["results"]["bindings"]:
  submorethan1dbvalue = int(el["tot"]["value"])

allsubsuridbvalue = int(allsubsuridbres["results"]["bindings"][0]["tot"]["value"])

intralinkingdb = submorethan1dbvalue/allsubsuridbvalue
print(intralinkingdb)



0.40480866915001695


## Europeana

In [ ]:
subjectmorethan1euro = '''
PREFIX ore: <http://www.openarchives.org/ore/terms/>
PREFIX edm: <http://www.europeana.eu/schemas/edm/>
SELECT (count(?sub) as ?tot) WHERE { FILTER (?tot > 1) {
select  ?sub (count(distinct ?CHO) as ?tot) where {
             ?s a <http://www.w3.org/2004/02/skos/core#Concept> ;
<http://www.w3.org/2004/02/skos/core#broader>* <http://vocab.getty.edu/aat/300033618> .
?s2 a <http://www.w3.org/2004/02/skos/core#Concept>  ;
<http://www.w3.org/2004/02/skos/core#broader>* <http://vocab.getty.edu/aat/300047090> .

{?CHO dc:type ?s;
dc:subject ?sub .}

UNION 

{?CHO dc:type ?s2;
dc:subject ?sub .}
}
GROUP BY ?sub }
}'''

allsubjectseuro = '''
PREFIX ore: <http://www.openarchives.org/ore/terms/>
PREFIX edm: <http://www.europeana.eu/schemas/edm/>
SELECT (count(distinct ?sub) as ?tot) WHERE {
             ?s a <http://www.w3.org/2004/02/skos/core#Concept> ;
<http://www.w3.org/2004/02/skos/core#broader>* <http://vocab.getty.edu/aat/300033618> .
?s2 a <http://www.w3.org/2004/02/skos/core#Concept>  ;
<http://www.w3.org/2004/02/skos/core#broader>* <http://vocab.getty.edu/aat/300047090> .

{?CHO dc:type ?s;
dc:subject ?sub .
FILTER (isURI(?sub))}

UNION 

{?CHO dc:type ?s2;
dc:subject ?sub .
FILTER(isURI(?sub))}
}'''

allseurores = sparl_query_setting(allsubjectseuro, euro)
allseuroresvalue = int(allseurores["results"]["bindings"][0]["tot"]["value"])
smorethan1eurores = sparl_query_setting(subjectmorethan1euro, euro)
smorethan1eurovalue = int(smorethan1eurores["results"]["bindings"][0]["tot"]["value"])

intralinkingeuro = smorethan1eurovalue/allseuroresvalue
print(intralinkingeuro)


0.6122093023255814


## Yago

In [ ]:
subjectmorethan1yago = '''
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX schema: <http://schema.org/>
SELECT (count(distinct ?sub) as ?tot) WHERE { FILTER (?tot > 1) {
SELECT ?sub (count(distinct ?artwork) as ?tot)
WHERE {
  VALUES ?art {schema:Painting schema:Sculpture}
  ?artwork rdf:type ?art ;
schema:about ?sub .
    } GROUP BY ?sub } }'''

allsubjectsyago = '''
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX schema: <http://schema.org/>
SELECT (count(distinct ?sub) as ?tot)
WHERE {
  VALUES ?art {schema:Painting schema:Sculpture}
  ?artwork rdf:type ?art ;
schema:about ?sub .
  FILTER (isURI(?sub))
    }'''

allsyagores = sparl_query_setting(allsubjectsyago, yago)
allsyagoresvalue = int(allsyagores["results"]["bindings"][0]["tot"]["value"])
smorethan1yagores = sparl_query_setting(subjectmorethan1yago, yago)
smorethan1yagovalue = int(smorethan1yagores["results"]["bindings"][0]["tot"]["value"])

intralinkingyago = smorethan1yagovalue/allsyagoresvalue
print(intralinkingyago)

0.16753536857781087


## ND Hungary

ND Hungary has no subjects or iconoraphic/iconologic related elements expressed by URIs, therefore its intra interlinking potential is 0

In [ ]:
iilpyndhungary = 0


## ARCO

In [ ]:
# number of subjects present in more than one artwork/tot. number of subj. 
intrainterlinkingpotentialarco = '''
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX arco-cd: <https://w3id.org/arco/ontology/context-description/>
PREFIX arco-d: <https://w3id.org/arco/ontology/denotative-description/>
PREFIX arco: <https://w3id.org/arco/ontology/arco/>

SELECT (COUNT(distinct ?sub) as ?tot)
WHERE { FILTER (?no > 1) {
 SELECT ?sub (COUNT(distinct ?artwork) AS ?no) 
 WHERE { ?artwork (arco-cd:hasSubject | arco-d:hasIconographicOrDecorativeApparatus) ?sub. 
?artwork a arco:HistoricOrArtisticProperty.}
GROUP BY ?sub }
}


  '''
iilpyarcores = sparl_query_setting(intrainterlinkingpotentialarco, arco_endpoint)

for el in iilpyarcores["results"]["bindings"]:
  subarcoscore = el["tot"]["value"]
  print(subpyarcoscore)

41947


In [ ]:
# tot num of subj
intrainterlinkingpotentialarco = '''
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX arco-cd: <https://w3id.org/arco/ontology/context-description/>
PREFIX arco-d: <https://w3id.org/arco/ontology/denotative-description/>
PREFIX arco: <https://w3id.org/arco/ontology/arco/>

SELECT (COUNT(distinct ?sub) as ?tot) where {
?artwork (arco-cd:hasSubject | arco-d:hasIconographicOrDecorativeApparatus) ?sub. 
?artwork a arco:HistoricOrArtisticProperty.

}


  '''
iilpyarcores = sparl_query_setting(intrainterlinkingpotentialarco, arco_endpoint)

for el in iilpyarcores["results"]["bindings"]:
  totarcoscore = el["tot"]["value"]
  print(totarcoscore)

242506


In [ ]:
s_intra = 41947/242506
print(s_intra)

0.1729730398423132


## Zeri

In [ ]:
# number of subjects present in more than one artwork/tot. number of subj. 
intrainterlinkingpotentialzeri = '''
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX oaentry: <http://purl.org/emmedi/oaentry/>
PREFIX crm: <http://www.cidoc-crm.org/cidoc-crm/>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX fabio: <http://purl.org/spar/fabio/>

SELECT (COUNT(distinct ?sub) as ?tot)
WHERE { FILTER (?no > 1) {
 SELECT ?sub (COUNT(distinct ?artwork) AS ?no) 
 WHERE { ?artwork fabio:hasSubjectTerm ?sub; a fabio:ArtisticWork.
 }
GROUP BY ?sub }
}


  '''
iilpyarcores = sparl_query_setting(intrainterlinkingpotentialzeri, zeri_endpoint)

for el in iilpyarcores["results"]["bindings"]:
  subzeriscore = el["tot"]["value"]
  print(subzeriscore)

1926


In [ ]:
# tot num of subj
intrainterlinkingpotentialzeri = '''
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX oaentry: <http://purl.org/emmedi/oaentry/>
PREFIX crm: <http://www.cidoc-crm.org/cidoc-crm/>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX fabio: <http://purl.org/spar/fabio/>


 SELECT (COUNT(distinct ?sub) AS ?tot) 
 WHERE { ?artwork fabio:hasSubjectTerm ?sub; a fabio:ArtisticWork.
 }

  '''
iilpyarcores = sparl_query_setting(intrainterlinkingpotentialzeri, zeri_endpoint)

for el in iilpyarcores["results"]["bindings"]:
  totzeriscore = el["tot"]["value"]
  print(totzeriscore)

s_intra = int(subzeriscore)/int(totzeriscore)
print(s_intra)

7232
0.2663163716814159


## Nomisma

In [ ]:
# number of subjects present in more than one artwork/tot. number of subj. 
intrainterlinkingpotentialnomisma = '''
PREFIX crm: <http://www.cidoc-crm.org/cidoc-crm/>
PREFIX dcmitype: <http://purl.org/dc/dcmitype/>
PREFIX dcterms: <http://purl.org/dc/terms/>
PREFIX foaf: <http://xmlns.com/foaf/0.1/>
PREFIX geo: <http://www.w3.org/2003/01/geo/wgs84_pos#>
PREFIX nm: <http://nomisma.org/id/>
PREFIX nmo: <http://nomisma.org/ontology#>
PREFIX org: <http://www.w3.org/ns/org#>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX skos: <http://www.w3.org/2004/02/skos/core#>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>

SELECT (COUNT(distinct ?sub) as ?tot)
WHERE { FILTER (?no > 1) {
 SELECT ?sub (COUNT(distinct ?artwork) AS ?no) 
 WHERE {    
   {?coin nmo:hasObverse ?artwork} UNION {?coin nmo:hasReverse ?artwork}
  ?artwork nmo:hasPortrait ?sub.
 }
GROUP BY ?sub }
}


  '''
res = sparl_query_setting(intrainterlinkingpotentialnomisma, nomisma_endpoint)

for el in res["results"]["bindings"]:
  subnomismascore = int(el["tot"]["value"])
  print(subnomismascore)

359


In [ ]:
# number of subjects present in more than one artwork/tot. number of subj. 
intrainterlinkingpotentialnomisma = '''
PREFIX crm: <http://www.cidoc-crm.org/cidoc-crm/>
PREFIX dcmitype: <http://purl.org/dc/dcmitype/>
PREFIX dcterms: <http://purl.org/dc/terms/>
PREFIX foaf: <http://xmlns.com/foaf/0.1/>
PREFIX geo: <http://www.w3.org/2003/01/geo/wgs84_pos#>
PREFIX nm: <http://nomisma.org/id/>
PREFIX nmo: <http://nomisma.org/ontology#>
PREFIX org: <http://www.w3.org/ns/org#>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX skos: <http://www.w3.org/2004/02/skos/core#>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>

SELECT (COUNT(distinct ?sub) as ?tot) 
 WHERE {    
   {?coin nmo:hasObverse ?artwork} UNION {?coin nmo:hasReverse ?artwork}
  ?artwork nmo:hasPortrait ?sub.
 }



  '''
res = sparl_query_setting(intrainterlinkingpotentialnomisma, nomisma_endpoint)

for el in res["results"]["bindings"]:
  totnomismascore = int(el["tot"]["value"])
  print(totnomismascore)
s_intra= subnomismascore/totnomismascore
print(s_intra)

479
0.7494780793319415


## Wikidata

In [ ]:
# number of subjects present in more than one artwork/tot. number of subj. 
intrainterlinkingpotentialwd1 = '''

SELECT (COUNT(distinct ?sub) as ?tot)
WHERE { FILTER (?no > 1) {
 SELECT ?sub (COUNT(distinct ?artwork) AS ?no) 
 WHERE {    
  {?artwork wdt:P31 wd:Q3305213} UNION {?artwork wdt:P31 wd:Q860861}
  ?artwork wdt:P180 | wdt:P921 | wdt:P1257 ?sub.
 }
GROUP BY ?sub }
}


  '''
res = sparl_query_setting(intrainterlinkingpotentialwd1, wd_endpoint)

for el in res["results"]["bindings"]:
  subwdscore = int(el["tot"]["value"])
  print(subwdscore)

intrainterlinkingpotentialwd2 = '''

SELECT (COUNT(distinct ?sub) as ?tot)
 WHERE {    
  {?artwork wdt:P31 wd:Q3305213} UNION {?artwork wdt:P31 wd:Q860861}
  ?artwork wdt:P180 | wdt:P921 | wdt:P1257 ?sub.
 }



  '''
res2 = sparl_query_setting(intrainterlinkingpotentialwd2, wd_endpoint)

for el in res2["results"]["bindings"]:
  totwdscore = int(el["tot"]["value"])
  print(totwdscore)

s_intra= subwdscore/totwdscore
print(s_intra)

14839
43195
0.3435351313809469


## SARI

In [ ]:
artwork_types= """<https://d-nb.info/gnd/4046277-8> <https://d-nb.info/gnd/4127900-1> <https://d-nb.info/gnd/4036042-8> <https://d-nb.info/gnd/4010427-8> <https://d-nb.info/gnd/4124432-1> <https://d-nb.info/gnd/4123412-1> <https://d-nb.info/gnd/4131808-0> <https://d-nb.info/gnd/4124432-1> <https://d-nb.info/gnd/4114333-4> <https://d-nb.info/gnd/4033760-1> <https://d-nb.info/gnd/4002511-1> <https://d-nb.info/gnd/4627787-0> <https://d-nb.info/gnd/4029783-4> <https://d-nb.info/gnd/4006627-7> <https://d-nb.info/gnd/4064542-3> <https://d-nb.info/gnd/4229053-3>"""

In [ ]:
# number of subjects present in more than one artwork/tot. number of subj. 
intrainterlinkingpotentialsari1 = """

SELECT (COUNT(distinct ?sub) as ?tot)
WHERE { FILTER (?no > 1) {
 SELECT ?sub (COUNT(distinct ?artwork) AS ?no) 
 WHERE {    
VALUES ?type {<https://d-nb.info/gnd/4046277-8> <https://d-nb.info/gnd/4127900-1> <https://d-nb.info/gnd/4036042-8> <https://d-nb.info/gnd/4010427-8> <https://d-nb.info/gnd/4124432-1> <https://d-nb.info/gnd/4123412-1> <https://d-nb.info/gnd/4131808-0> <https://d-nb.info/gnd/4124432-1> <https://d-nb.info/gnd/4114333-4> <https://d-nb.info/gnd/4033760-1> <https://d-nb.info/gnd/4002511-1> <https://d-nb.info/gnd/4627787-0> <https://d-nb.info/gnd/4029783-4> <https://d-nb.info/gnd/4006627-7> <https://d-nb.info/gnd/4064542-3> <https://d-nb.info/gnd/4229053-3>}
  ?artwork gndo:formOfWorkAndExpression ?type.
   ?artwork  gndo:topic | gndo:gndSubjectCategory ?sub.
 }
GROUP BY ?sub }
}
  
"""
# res: 9

res = sparl_query_setting(intrainterlinkingpotentialsari1, sari_endpoint)

for el in res["results"]["bindings"]:
  subsariscore = int(el["tot"]["value"])
  print(subsariscore)

intrainterlinkingpotentialsari2 = '''

SELECT (COUNT(distinct ?sub) as ?tot)
 WHERE {    
VALUES ?type {'''+artwork_types+'''}
  ?artwork gndo:formOfWorkAndExpression ?type.
   ?artwork  (gndo:topic | gndo:gndSubjectCategory) ?sub.
 }



  '''
res2 = sparl_query_setting(intrainterlinkingpotentialsari2, sari_endpoint)
# res: 18 
for el in res2["results"]["bindings"]:
  totsariscore = int(el["tot"]["value"])
  print(totwdscore)

s_intra= subsariscore/totsariscore
print(s_intra)
# s_intra = 0,5

EndPointInternalError: ignored